In [ ]:
!pip install unrar
!unrar x "/content/recordings.rar"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/recordings.rar

Creating    recordings_filtered                                       OK
Creating    recordings_filtered/1                                     OK
Extracting  recordings_filtered/1/column1.txt                              0%  OK 
Extracting  recordings_filtered/1/column3.txt                              1%  OK 
Extracting  recordings_filtered/1/metadata.txt                             1%  OK 
Creating    recordings_filtered/10                                    OK
Extracting  recordings_filtered/10/column1.txt                             2%  OK 
Extracting  recordings_filtered/10/column3.txt                             3%  OK 
Extracting  recordings_filtered/10/metadata.txt                            3%  OK 
Creating    recordings_filtered/11                                    OK
Extracting  recordings_filtered/11/colu

In [ ]:
import os
from datetime import datetime
import numpy as np
import numpy as np
import pandas as pd
from scipy import stats
from scipy.fft import fft
from scipy.signal import find_peaks
from itertools import groupby

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Patient:
    def __init__(self, id, name, surname, test_date, breath_sensor, audio_sensor, is_sick):
        self.id = id  # Hasta numarası
        self.name = name  # Hastanın adı
        self.surname = surname  # Hastanın soyadı
        self.test_date = test_date  # Test tarihi (datetime objesi)
        self.breath_sensor = breath_sensor  # column1.txt verisi: integer listesi
        self.audio_sensor = audio_sensor  # column3.txt verisi: double dizisi (list of floats)
        self.is_sick = is_sick

   # def __repr__(self):
       # return (f"Id={self.id},Name={self.name}, Surname={self.surname}, Test_Date={self.test_date}, "
        #        f"breath_sensor={self.breath_sensor}, audio_sensor={self.audio_sensor}, is_patient={self.is_sick}")

    def __repr__(self):
       return (f"Id={self.id} "
                f"breath_sensor={self.breath_sensor}, is_patient={self.is_sick}")

In [ ]:
# 'recordings_filtered' klasörünün yolu
base_dir = '/content/recordings_filtered'

# Hasta nesnelerini saklamak için bir liste
patient_list = []

# 1'den 50'ye kadar olan klasörlerde döngü
for i in range(1, 51):
    folder_path = os.path.join(base_dir, str(i))  # Her bir klasörün yolu

    # column1.txt, column3.txt ve hasta bilgilerini içeren dosya yolları
    column1_path = os.path.join(folder_path, 'column1.txt')
    column3_path = os.path.join(folder_path, 'column3.txt')

    # Hasta bilgilerini içeren metadata.txt dosyasının yolu
    metadata_path = os.path.join(folder_path, 'metadata.txt')

    try:
        with open(column1_path, 'r') as file1, open(column3_path, 'r') as file3, open(metadata_path, 'r') as meta_file:

            id = i

            if(i<=35):
              is_sick = False
            else:
              is_sick = True

            # column1.txt içeriğini integer listesine çevir
            breath_sensor = list(map(int, file1.read().strip()[1:-1].split('\n')))  # [2,3,5,9,0,7] formatında

            # column3.txt içeriğini bir double dizisine çevir
            audio_sensor = list(map(float, file3.read().strip().split()))

            # metadata.txt dosyasından hasta bilgilerini al (ad, soyad, test tarihi)
            metadata_lines = meta_file.read().strip().split('\n')

            # Patient Name: John Doe satırından adı al
            name_surname = metadata_lines[0].replace("Patient Name: ", "")
            name, surname = name_surname.split(" ", 1)  # Ad ve soyadı ayır

            # Date: 2025-03-22_16-43-37 satırından test tarihini al
            test_date_str = metadata_lines[1].replace("Date: ", "")
            # Test tarihini datetime objesine dönüştür (Date: 2025-03-22_16-43-37 formatı)
            test_date = datetime.strptime(test_date_str, '%Y-%m-%d_%H-%M-%S')

            # Yeni Hasta nesnesini oluştur
            patient = Patient(id, name, surname, test_date, breath_sensor, audio_sensor, is_sick)

            # Hasta nesnesini listeye ekle
            patient_list.append(patient)
    except FileNotFoundError:
        print(f"Dosya bulunamadı: {folder_path}")
        continue

# Listeyi görüntüle (isteğe bağlı)
for patient in patient_list:
    print(patient)

Id=1 breath_sensor=[95, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4027, 3864, 3393, 3237, 2919, 2910, 2769, 2749, 2509, 2352, 2051, 1681, 2098, 2495, 2969, 3347, 4022, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 3939, 3550, 2742, 1935, 1538, 987, 900, 1244, 1433, 1352, 1823, 1903, 1727, 1799, 1802, 1635, 2032, 2671, 3217, 3693, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 3455, 2644, 1829, 1011, 193, 134, 721, 971, 1314, 1522, 1608, 1189, 1599, 2074, 2683, 3296, 3939, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 3788, 3070, 2282, 1473, 654, 142, 55, 31, 25, 43, 62, 75, 487, 1301, 2102, 2903, 3693, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 409], is_patient=False
Id=2 breath_sensor=[95, 4095, 40

In [ ]:
def extract_basic_features(sensor_data):
    """Temel istatistiksel özellikleri çıkarır"""
    return {
        'mean': np.mean(sensor_data),
        'std_dev': np.std(sensor_data),
        'min': np.min(sensor_data),
        'range': np.ptp(sensor_data),
        'median': np.median(sensor_data),
        'skewness': stats.skew(sensor_data),
        'kurtosis': stats.kurtosis(sensor_data),
        'q1': np.percentile(sensor_data, 25),
        'q3': np.percentile(sensor_data, 75),
        'iqr': stats.iqr(sensor_data),
    }

In [ ]:
def extract_breath_features(sensor_data, max_val=4095):
    """Nefes dalgasına özgü özellikleri çıkarır"""
    # Convert sensor_data to a NumPy array
    sensor_data = np.array(sensor_data)

    peaks, _ = find_peaks(sensor_data, height=max_val*0.8, distance=10)
    troughs, _ = find_peaks(max_val - sensor_data, height=max_val*0.8, distance=10)

    peak_values = sensor_data[peaks] if len(peaks) > 0 else [0]
    trough_values = sensor_data[troughs] if len(troughs) > 0 else [0]

    return {
        'num_peaks': len(peaks),
        'num_troughs': len(troughs),
        'avg_peak_height': np.mean(peak_values),
        'avg_trough_depth': np.mean(trough_values),
        'breath_amplitude': np.mean(peak_values) - np.mean(trough_values) if len(peaks)>0 and len(troughs)>0 else 0,
        'peak_variability': np.std(peak_values) if len(peaks) > 1 else 0,
        'trough_variability': np.std(trough_values) if len(troughs) > 1 else 0,
        'max_cluster': max(len(list(g)) for k, g in groupby(sensor_data == max_val)) if max_val in sensor_data else 0,
    }

In [ ]:
def extract_time_series_features(sensor_data):
    """Zaman serisi analizi özellikleri"""
    n = len(sensor_data)
    yf = fft(sensor_data) # Call the imported fft function
    power_spectrum = np.abs(yf[:n//2])**2

    return {
        'entropy': stats.entropy(np.histogram(sensor_data, bins=20)[0]+1e-10),
        'zero_crossings': len(np.where(np.diff(np.sign(sensor_data - np.mean(sensor_data))))[0]),
        'autocorr_lag1': np.corrcoef(sensor_data[:-1], sensor_data[1:])[0,1],
        'dominant_freq': np.argmax(power_spectrum) if len(power_spectrum) > 0 else 0,
        'spectral_energy': np.sum(power_spectrum),
    }

In [ ]:
def extract_special_features(sensor_data, max_val=4095):
    """Hastalıkla ilişkili özel özellikler"""
    max_val_stuck = sum(1 for x in sensor_data if x == max_val)
    diffs = np.diff(sensor_data)
    zero_diffs = sum(1 for d in diffs if d == 0)

    return {
        'max_val_percentage': max_val_stuck / len(sensor_data),
        'zero_diff_percentage': zero_diffs / len(diffs),
        'plateau_count': sum(1 for k, g in groupby(sensor_data)) / len(sensor_data),
        'abrupt_changes': sum(1 for d in diffs if abs(d) > max_val*0.3) / len(diffs),
    }

In [ ]:
def extract_all_features(patient):
    """Bir Patient nesnesinden tüm özellikleri çıkarır"""
    features = {}

    # Nefes sensörü özellikleri
    breath_data = patient.breath_sensor
    features.update(extract_basic_features(breath_data))
    features.update(extract_breath_features(breath_data))
    features.update(extract_time_series_features(breath_data))
    features.update(extract_special_features(breath_data))

    return features

In [ ]:
features = []
labels = []

for patient in patient_list:
    features.append(extract_all_features(patient))
    labels.append(patient.is_sick)

df = pd.DataFrame(features)
X = df.values
y = np.array(labels)

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df

,mean,std_dev,min,range,median,skewness,kurtosis,q1,q3,iqr,num_peaks,num_troughs,avg_peak_height,avg_trough_depth,breath_amplitude,peak_variability,trough_variability,max_cluster,entropy,zero_crossings,autocorr_lag1,dominant_freq,spectral_energy,max_val_percentage,zero_diff_percentage,plateau_count,abrupt_changes
0,3121.735849,1328.131522,25,4070,4095.0,-1.044728,-0.324315,2062.50,4095.0,2032.50,5,2,4095.000000,79.500000,4015.500000,0.000000,54.500000,21,1.889867,10,0.915055,0,2.686663e+11,0.540881,0.512658,0.490566,0.012658
1,3025.272727,1615.783708,5,4090,4095.0,-1.045474,-0.708875,1827.50,4095.0,2267.50,4,3,4095.000000,24.666667,4070.333333,0.000000,15.542058,32,1.471935,8,0.940759,0,2.480137e+11,0.629870,0.620915,0.383117,0.013072
2,2747.643312,1706.886557,0,4095,4095.0,-0.698186,-1.307849,895.00,4095.0,3200.00,5,4,4095.000000,6.000000,4089.000000,0.000000,4.636809,23,1.628708,10,0.941718,0,2.219956e+11,0.528662,0.500000,0.503185,0.012821
3,2799.780000,1685.624287,2,4093,4095.0,-0.758843,-1.204531,1001.75,4095.0,3093.25,5,4,4095.000000,12.750000,4082.250000,0.000000,10.425330,21,1.597120,10,0.935653,0,2.083370e+11,0.540000,0.536913,0.466667,0.013423
4,3004.493506,1633.066998,3,4092,4095.0,-1.014784,-0.781829,1689.50,4095.0,2405.50,4,3,4095.000000,31.333333,4063.666667,0.000000,31.329787,28,1.454193,8,0.940787,0,2.457080e+11,0.623377,0.607843,0.396104,0.013072
5,2746.155556,1684.371188,8,4087,4095.0,-0.676000,-1.323898,835.50,4095.0,3259.50,6,5,4095.000000,37.000000,4058.000000,0.000000,28.802778,23,1.789162,12,0.941229,0,2.903007e+11,0.516667,0.497207,0.505556,0.011173
6,2993.880240,1497.801302,26,4069,4095.0,-0.881822,-0.918537,1389.50,4095.0,2705.50,5,4,4095.000000,269.000000,3826.000000,0.000000,219.510820,25,1.772434,10,0.932054,0,2.812601e+11,0.562874,0.542169,0.461078,0.012048
7,3217.970414,1415.645977,95,4000,4095.0,-1.221619,-0.243617,2477.00,4095.0,1618.00,4,3,4095.000000,214.000000,3881.000000,0.000000,82.957821,34,1.467119,8,0.933727,0,3.243774e+11,0.650888,0.630952,0.372781,0.011905
8,3353.085227,1237.346643,63,4032,4095.0,-1.449967,0.653640,2784.75,4095.0,1310.25,5,3,4095.000000,340.333333,3754.666667,0.000000,213.699685,28,1.550071,10,0.909233,0,3.719808e+11,0.647727,0.622857,0.380682,0.011429
9,3135.217143,1444.307351,36,4059,4095.0,-1.145208,-0.355286,2140.50,4095.0,1954.50,5,3,4095.000000,76.333333,4018.666667,0.000000,54.236109,26,1.642338,10,0.930827,0,3.329726e+11,0.577143,0.563218,0.440000,0.011494


from matplotlib import pyplot as plt
df['mean'].plot(kind='hist', bins=20, title='mean')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['std_dev'].plot(kind='hist', bins=20, title='std_dev')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['min'].plot(kind='hist', bins=20, title='min')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['range'].plot(kind='hist', bins=20, title='range')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['q3']
  ys = series['range']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df.sort_values('q3', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('q3')
_ = plt.ylabel('range')

from matplotlib import pyplot as plt
df['mean'].plot(kind='line', figsize=(8, 4), title='mean')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['std_dev'].plot(kind='line', figsize=(8, 4), title='std_dev')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['min'].plot(kind='line', figsize=(8, 4), title='min')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['range'].plot(kind='line', figsize=(8, 4), title='range')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
!cp -r "/content/recordings_filtered" "/content/drive/MyDrive/bitirmeProjesi/"